<a href="https://colab.research.google.com/github/rodrigopbbr/aprendizado_de_maquina/blob/main/id3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# biblioteca para manipulação de dados
import pandas as pd
# biblioteca para operações matemáticas
import numpy as np
# biblioteca para operações aleatórias
# biblioteca para carregar arquivos (e.g. csv)
from google.colab import files
from collections import Counter

In [ ]:
files.upload()

Saving jogar-tenis-2.csv to jogar-tenis-2.csv


{'jogar-tenis-2.csv': b'Tempo;Temperatura;Umidade;Vento;Jogar\nEnsolarado;Quente;Alta;Fraco;N\xc3\xa3o\nEnsolarado;Quente;Alta;Forte;N\xc3\xa3o\nNublado;Quente;Alta;Fraco;Sim\nChuvoso;Intermedi\xc3\xa1ria;Alta;Fraco;Sim\nChuvoso;Fria;Normal;Fraco;Sim\nChuvoso;Fria;Normal;Forte;N\xc3\xa3o\nNublado;Fria;Normal;Forte;Sim\nEnsolarado;Intermedi\xc3\xa1ria;Alta;Fraco;N\xc3\xa3o\nEnsolarado;Fria;Normal;Fraco;Sim\nChuvoso;Intermedi\xc3\xa1ria;Normal;Fraco;Sim\nEnsolarado;Intermedi\xc3\xa1ria;Normal;Forte;Sim\nNublado;Intermedi\xc3\xa1ria;Alta;Forte;Sim\nNublado;Quente;Normal;Fraco;Sim\nChuvoso;Intermedi\xc3\xa1ria;Alta;Forte;N\xc3\xa3o\n'}

In [ ]:
# carregando o conjunto de dados iris (https://pt.wikipedia.org/wiki/Conjunto_de_dados_flor_Iris)
dataset = pd.read_csv('jogar-tenis-2.csv', sep=";")
# visualizando parte dos dados
dataset

,Tempo,Temperatura,Umidade,Vento,Jogar
0,Ensolarado,Quente,Alta,Fraco,Não
1,Ensolarado,Quente,Alta,Forte,Não
2,Nublado,Quente,Alta,Fraco,Sim
3,Chuvoso,Intermediária,Alta,Fraco,Sim
4,Chuvoso,Fria,Normal,Fraco,Sim
5,Chuvoso,Fria,Normal,Forte,Não
6,Nublado,Fria,Normal,Forte,Sim
7,Ensolarado,Intermediária,Alta,Fraco,Não
8,Ensolarado,Fria,Normal,Fraco,Sim
9,Chuvoso,Intermediária,Normal,Fraco,Sim


In [ ]:
# separando o rótulo Species (Aprendizado de Máquina Supervisionado) dos demais atributos: y
y = dataset['Jogar']
dataset = dataset.drop(['Jogar'], axis = 1)

In [ ]:
class ID3:
  def __init__(self, att):        
    self.atributo = att
    self.filhos = []
    self.arestas = []

  def add_filho(self, f):
    self.filhos.append(f)

  def add_aresta(self, a):
    self.arestas.append(a)

  def get_arestas(self):
    return self.arestas

  def get_filhos(self):
    return self.filhos

  def get_atributo(self):
    return self.atributo

  def set_filhos(self, f):
    self.filhos = f

  def set_arestas(self, a):
    self.arestas = a

  def print_filhos(self):
    for i in self.filhos:
      print(f"{i.get_atributo()}")

  def print_arestas(self):
    print(f"{self.arestas}")

  def print_arvore(self, tab):
    print("|", end='')
    for i in np.arange(tab):
      print("__", end='')
    print(f"{self.get_atributo()}")
    for i in np.arange(len(self.get_filhos())):
      self.get_filhos()[i].print_arvore(tab=tab+1)

In [ ]:
def entropia (valor):
  ent = 0
  if 0 in valor:
    valor.remove(0)
  if len(valor) > 1:
    p = np.array(valor)/np.sum(valor)
    ent = np.sum(p*np.log2(p))*(-1)
  return ent


In [ ]:
def ganho_informacao(atributo, classe):
  valores = list(Counter(atributo).keys())
  ganho = []
  for i in np.arange(len(valores)):    
    sv = list(Counter(classe.iloc[np.where(atributo == valores[i])]).values())
    ganho.append(entropia(sv)*np.sum(sv)/len(classe))
  return entropia(list(Counter(classe).values()))-np.sum(ganho)


In [ ]:
def fit(X, y, min = 1):

  if len(y) <= min:
    id3 = ID3(max(Counter(y)))
  elif len(Counter(y)) == 1:
    id3 = ID3(list(Counter(y).keys())[0])
  else:
    ganhos = []
    for i in np.arange(X.shape[1]):
      ganhos.append(ganho_informacao(X.iloc[:,i], y))

    best_att = np.argmax(ganhos)
    id3 = ID3(X.columns[best_att])
    id3.set_arestas(list(Counter(X.iloc[:,best_att]).keys()))

    for i in id3.get_arestas():
      X_sub = X.iloc[np.where(X.iloc[:, best_att] == i)]
      X_sub = X_sub.drop(X_sub.columns[best_att], axis = 1) 
      y_sub = y.iloc[np.where(X.iloc[:, best_att] == i)]   
      
      id3.add_filho(fit(X_sub, y_sub))

  return id3

In [ ]:
def predict(arvore, consulta):
  if len(arvore.get_filhos()) == 0:
    print(f"{arvore.get_atributo()}")
  else:
    filho = arvore.get_arestas().index(consulta[arvore.get_atributo()])
    predict(arvore.get_filhos()[filho], consulta)


In [ ]:
clf = fit(dataset, y)

In [ ]:
clf.print_arvore(1)

|__Tempo
|____Umidade
|______Não
|______Sim
|____Sim
|____Vento
|______Sim
|______Não


In [ ]:
# Tempo = ens	Temperatura	Umidade	Vento
consulta = dataset.iloc[0]

consulta['Tempo'] = 'Ensolarado'
consulta['Temperatura'] = 'Quente'
consulta['Umidade'] = 'Normal'
consulta['Vento'] = 'Fraco'

#print(f"{consulta}")
print("---predição---")
predict(clf, consulta)

---predição---
Sim


In [ ]:
consulta['Tempo'] = 'Ensolarado'
consulta['Temperatura'] = 'Quente'
consulta['Umidade'] = 'Alta'
consulta['Vento'] = 'Fraco'

#print(f"{consulta}")
print("---predição---")
predict(clf, consulta)

---predição---
Não


In [ ]:
consulta['Tempo'] = 'Nublado'
consulta['Temperatura'] = 'Quente'
consulta['Umidade'] = 'Normal'
consulta['Vento'] = 'Fraco'

#print(f"{consulta}")
print("---predição---")
predict(clf, consulta)

---predição---
Sim


In [ ]:
consulta['Tempo'] = 'Chuvoso'
consulta['Temperatura'] = 'Quente'
consulta['Umidade'] = 'Normal'
consulta['Vento'] = 'Fraco'

#print(f"{consulta}")
print("---predição---")
predict(clf, consulta)

---predição---
Sim


In [ ]:
consulta['Tempo'] = 'Chuvoso'
consulta['Temperatura'] = 'Quente'
consulta['Umidade'] = 'Normal'
consulta['Vento'] = 'Forte'

#print(f"{consulta}")
print("---predição---")
predict(clf, consulta)

---predição---
Não
